# A FICO problem

Fico is the industry standard for determining credit worthiness. However, research shows that FICO is overused. FICO's usage fits the idiom "square peg in a round hole".

Evidence:

* A [research study](https://www.stlouisfed.org/publications/regional-economist/october-2008/did-credit-scores-predict-the-subprime-crisis) by the Federal Reserve Bank of St. Louis demonstrates that __FICO scores were a poor indicater of the subprime mortgage crisis__.
> Given the nature of FICO scores, one might expect to find a relationship between borrowers’ scores and the incidence of default and foreclosure.....FICO scores have not indicated that relationship....higher FICO scores have been associated with bigger increases in default rates over time.
* [Studies prove](https://www.marketwatch.com/story/your-digital-footprint-could-provide-a-more-accurate-credit-score-2018-05-03) that __digital footprints can outperform FICO__.
> Those who order from mobile phones are three times as likely to default as those who order from desktops. A customer who uses her name in her email address is 30 percent less likely to default than one who doesn’t. Those who shop between noon and 6 p.m. are half as likely to default as midnight to 6 a.m. buyers
* FICO depends on credit usage (CU). __CU is unreliable__ because it fluctuates through economic cycles. [CU amongs millenials dropped](http://fortune.com/2018/02/27/why-millennials-are-ditching-credit-cards/), especially during the 2008 recession. But being debt conscious is financially desirable.
* Paying timely minimums on credit statements results in both a strong score and __mounting debt__.
* Credit utilization ratios factor heavily into FICO ( $util = \frac{debt}{availCredt}$ ). A healthy utilization is < 0.3. Credit card sneak companies sneakily encourage opening several credit lines to improve this ratio. There is a conflict of interest, because __they profit on credit debt__.

# An ML solution to credit defaults

30,000 instances of credit default data were collected by the University of California Irvine.

In [2]:
# Data Exploration

import pandas as pd
fname = './UCI_Credit_Card.csv'

df = pd.read_csv(fname)
df = df.drop(['ID'], axis=1)
df.sample(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
3227,190000.0,2,1,2,32,-2,-2,-2,-2,-2,...,15010.0,4575.0,7279.0,922.0,3208.0,15010.0,4575.0,7279.0,0.0,0
3942,60000.0,2,2,2,29,2,-1,-1,2,0,...,58698.0,58033.0,57539.0,59068.0,4898.0,0.0,2300.0,2200.0,2110.0,0
24010,90000.0,1,2,2,25,0,0,0,0,0,...,52285.0,46430.0,47404.0,3453.0,3868.0,1960.0,1658.0,1718.0,2200.0,0
29414,130000.0,1,2,2,28,0,0,0,0,0,...,76133.0,77648.0,79370.0,3000.0,2709.0,2750.0,2767.0,2961.0,2913.0,0
2475,30000.0,2,3,2,25,0,0,0,0,2,...,29139.0,28534.0,30360.0,1500.0,1447.0,3000.0,0.0,2300.0,0.0,0


In [3]:
from time import time
import numpy as np
from sklearn import svm, preprocessing
from sklearn.metrics import accuracy_score
from read_csv import read

def main():
    data_set = read(fname)
    features, labels = split_features_labels(data_set)
    train_features, train_labels, test_features, test_labels = split_train_test(features, labels, 0.7)
    print(len(train_features), ' ', len(test_features))
    clf = svm.SVC()
    print('Start training...')
    tStart = time()
    clf.fit(train_features, train_labels)
    print('Training time: ', round(time()-tStart, 3), 's')
    print('Accuracy: ', accuracy_score(clf.predict(test_features), test_labels))

def split_train_test(features, labels, test_size):
    total_test_size = int(len(features) * test_size)
    np.random.seed(2)
    indices = np.random.permutation(len(features))
    train_features = features[indices[:-total_test_size]]
    train_labels = labels[indices[:-total_test_size]]
    test_features  = features[indices[-total_test_size:]]
    test_labels  = labels[indices[-total_test_size:]]
    return train_features, train_labels, test_features, test_labels

def split_features_labels(data_set):
    features = data_set['features']
    labels = data_set['labels']

    ids_removed = [x[1:] for x in features]
    features = [x[:11] for x in ids_removed]

    bills = [x[11:17] for x in ids_removed]
    payments = [x[17:] for x in ids_removed]

    div = lambda a,b: [x/y if y > 0 else 1 for x, y in zip(a, b)]
    fraction_paid = list(map(div, payments, bills))

    features = [val+fraction_paid[idx] for idx,val in enumerate(features)]

    features = np.array([np.array(x) for x in features])
    labels = np.array(labels)

    return preprocessing.scale(features), labels

if __name__ == '__main__':
    main()

9000   21000
Start training...
Training time:  2.951 s
Accuracy:  0.8194285714285714
